# 第6章 逻辑斯谛回归与最大熵模型
## 6.1 逻辑斯谛回归
逻辑斯谛回归模型假定随机变量X的数据分布符合逻辑斯谛回归,有:  
>逻辑斯谛回归分布函数：
$$F(x)=P(X \leq x)=\frac{1}{1+e^\left(-\frac{x-\mu}{\gamma}\right)}$$

>密度函数：
$$f(x)=F'(x)=\frac{e^\left(-\frac{x-\mu}{\gamma}\right)}{\gamma(1+e^\left(-\frac{x-\mu}{\gamma}\right))^2}$$